In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

#### Dicionário dos Dados


In [2]:
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")

df_dict

,variavel,descricao,tipo,subtipo
0,species,Espécie de ave,qualitativa,nominal
1,island,Ilha onde a ave foi observada,qualitativa,nominal
2,bill_length_mm,Comprimento do bico de ave em milímetros,quantitativa,contínua
3,bill_depth_mm,Profundidade do bico da ave em milímetros,quantitativa,contínua
4,flipper_length_mm,Comprimento da nadadeira da ave em milímetros,quantitativa,contínua
5,body_mass_g,Massa corporal da ave em gramas,quantitativa,contínua
6,sex,Sexo da ave (Male ou Female).,qualitativa,nominal


In [3]:
## Contagem de valores ausentes.
print("\nValores ausentes em cada coluna:")
display(df.isnull().sum())
print("\n")

# Exibe os tipos de cada coluna.
print("Tipos de dados de cada coluna:")
print(df.dtypes)

# Filtra as linhas duplicadas e conta a quantidade.
duplicate = df[df.duplicated()]
num_duplicate = duplicate.shape[0]

# Exibe as linhas duplicadas e a contagem.
print(f"\nNúmero total de linhas duplicadas: {num_duplicate}")

# Remove as linhas duplicadas do dataset
df.drop_duplicates().reset_index(drop = True)

df_dict


Valores ausentes em cada coluna:


species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64



Tipos de dados de cada coluna:
species               object
island                object
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                   object
dtype: object

Número total de linhas duplicadas: 0


,variavel,descricao,tipo,subtipo
0,species,Espécie de ave,qualitativa,nominal
1,island,Ilha onde a ave foi observada,qualitativa,nominal
2,bill_length_mm,Comprimento do bico de ave em milímetros,quantitativa,contínua
3,bill_depth_mm,Profundidade do bico da ave em milímetros,quantitativa,contínua
4,flipper_length_mm,Comprimento da nadadeira da ave em milímetros,quantitativa,contínua
5,body_mass_g,Massa corporal da ave em gramas,quantitativa,contínua
6,sex,Sexo da ave (Male ou Female).,qualitativa,nominal


#### Preparação dos Dados

In [4]:
target_variable = 'species'

nominal_variables = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)

continuous_variables = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

x = df.drop(columns=[target_variable], axis=1)
y = df[target_variable]

In [5]:
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')), # tratamento para dados faltantes
    ("encoding", OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])

continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), 
    ('normalization', StandardScaler()) 
])

In [6]:
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_variables),
    ('continuous', continuous_preprocessor, continuous_variables)
])


In [7]:
preprocessor

ColumnTransformer(transformers=[('nominal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False)),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['island', 'sex']),
                                ('continuous',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 [])])